# Manipulate a function
Tutorial by Jonas Wilfert, Tobias Thummerer

## License
Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert

Licensed under the MIT license. See [LICENSE](https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multithreading**, check `distributed.ipynb` for multiprocessing.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/main/example/parallel.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/main/example/parallel.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/parallel.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install Folds via                 | add Folds                 |                                                |
| 6.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section

To run the example, the previously installed packages must be included. 

In [6]:
# imports
using FMI
using FMIZoo
using Folds
using BenchmarkTools

Checking the amount of threads:

In [7]:
Threads.nthreads()

16

### Simulation setup

Next, the start time and end time of the simulation are set. Here we also decide the size of the batch.

In [8]:
t_start = 0.0
t_step = 0.1
t_stop = 10.0
tspan = (t_start, t_stop)
tData = collect(t_start:t_step:t_stop)

# Best if batchSize is a multiple of the threads/cores
batchSize = 16

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))


16-element Vector{Vector{Float64}}:
 [0.7570142201163749, 0.7816539687562132]
 [0.4445385856868076, 0.7130175277113704]
 [0.11226398863967602, 0.4748508019060218]
 [0.808954051155261, 0.27434471724011544]
 [0.30293560141225895, 0.3016936728488645]
 [0.7883805568751233, 0.8819293061570325]
 [0.613293613060242, 0.7822095031551625]
 [0.2601160763007673, 0.3113685929125576]
 [0.020134856959976632, 0.671682472913182]
 [0.26685478653841466, 0.7076480324355322]
 [0.9346519795934783, 0.8965378205666846]
 [0.9892106345675635, 0.3200750058562988]
 [0.21365467385900394, 0.32398921105913403]
 [0.6032901394162044, 0.7327809415221315]
 [0.14019227948592672, 0.19653698600205638]
 [0.9844595020558117, 0.7579371060757065]

We need to instantiate one FMU for each parallel execution, as they cannot share state.

In [ ]:
realFMU = fmiLoad("SpringPendulum1D", "Dymola", "2022x")
fmiInstantiate!(realFMU)


realFMUBatch = [fmiLoad("SpringPendulum1D", "Dymola", "2022x") for _ in 1:batchSize]
fmiInstantiate!.(realFMUBatch)

We define a helper function to calculate the FMU and combine it into an Matrix.

In [10]:
function runCalcFormatted(fmu::FMU2, x0::Vector{Float64}, recordValues::Vector{String}=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, t_start, t_stop; recordValues=recordValues, saveat=tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

runCalcFormatted (generic function with 2 methods)

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [11]:
@benchmark data = runCalcFormatted(realFMU, rand(2))

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  317.244 ms … 336.055 ms  ┊ GC (min … max): 2.32% … 2.45%
 Time  (median):     327.519 ms               ┊ GC (median):    4.14%
 Time  (mean ± σ):   327.401 ms ±   4.562 ms  ┊ GC (mean ± σ):  3.72% ± 0.91%

  ▁               ▁    ▁▁  ▁ ▁  ▁▁   ▁▁█  ▁      ▁       ▁    ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁██▁▁█▁█▁▁██▁▁▁███▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁█ ▁
  317 ms           Histogram: frequency by time          336 ms <

 Memory estimate: 119.32 MiB, allocs estimate: 3002344.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [12]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(realFMU, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 5.193 s (3.67% GC) to evaluate,
 with a memory estimate of 1.86 GiB, over 48037492 allocations.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple threads, the library `Folds` can be used.

In [13]:
println("Multi Threaded")
@benchmark Folds.collect(runCalcFormatted(fmu, i) for (fmu, i) in zip(realFMUBatch, input_values))

Multi Threaded


BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.356 s …    1.807 s  ┊ GC (min … max): 37.10% … 52.35%
 Time  (median):     1.630 s               ┊ GC (median):    47.77%
 Time  (mean ± σ):   1.606 s ± 190.432 ms  ┊ GC (mean ± σ):  46.80% ±  6.48%

  █                           █            █               █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.36 s         Histogram: frequency by time         1.81 s <

 Memory estimate: 1.86 GiB, allocs estimate: 48037692.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [14]:
fmiUnload(realFMU)
fmiUnload.(realFMUBatch)

16-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

### Summary

In this tutorial it is shown how multi threading with `Folds.jl` can be used to improve the performance for calculating a Batch of FMUs.